In [1]:
import csv
import numpy as np
import pandas as pd
import sys, os
import re
import ast
import datetime as dt
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
diag = pd.read_csv('../data/hos/diagnoses_icd.csv').reset_index(drop = True)
# diag = diag.reset_index(inplace=True)
diag

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4756321,19999987,23865745,7,41401,9
4756322,19999987,23865745,8,78039,9
4756323,19999987,23865745,9,0413,9
4756324,19999987,23865745,10,36846,9


In [21]:
diag['icd_version'].value_counts()
# diag.shape #(4756326, 6)

9     2766877
10    1989449
Name: icd_version, dtype: int64

In [3]:
diag['new'] = diag['icd_code']

In [24]:
#adding underscore with the ICD version on the ICD_codes
diag.loc[diag["icd_version"] == 9, "new"] = '9_' + diag.loc[diag["icd_version"] == 9, "new"]
diag.loc[diag["icd_version"] == 10, "new"] = '10_' + diag.loc[diag["icd_version"] == 10, "new"]


In [25]:
diag.shape
diag.loc[diag["icd_version"] == 10]

,subject_id,hadm_id,seq_num,icd_code,icd_version,new
40,10000084,23052089,1,G3183,10,10_G3183
41,10000084,23052089,2,F0280,10,10_F0280
42,10000084,23052089,3,R441,10,10_R441
43,10000084,23052089,4,R296,10,10_R296
44,10000084,23052089,5,E785,10,10_E785
...,...,...,...,...,...,...
4756291,19999828,29734428,18,Z9049,10,10_Z9049
4756292,19999828,29734428,19,Z87891,10,10_Z87891
4756293,19999828,29734428,20,B9620,10,10_B9620
4756294,19999828,29734428,21,Z1611,10,10_Z1611


In [26]:
d_diag = pd.read_csv('../data/hos/d_icd_diagnoses.csv').reset_index(drop = True)


# diag = diag.reset_index(inplace=True)
d_diag #109775 rows × 3 columns

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [30]:
d_diag.loc[d_diag['icd_version']==9]['icd_code'].nunique()#there are 14666 unique ICD code in version 9
d_diag.loc[d_diag['icd_version']==10]['icd_code'].nunique() #there are 95109 unique ICD code in version 10
diag.loc[diag['icd_version']==9]['icd_code'].nunique()#there are 9072 unique ICD code in version 9
diag.loc[diag['icd_version']==10]['icd_code'].nunique() #there are 16757 unique ICD code in version 10

16757

In [6]:
(diag
 .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version'])
.reset_index())

,index,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title
0,0,10000032,22595853,1,5723,9,5723,Portal hypertension
1,1,10000826,20032235,4,5723,9,5723,Portal hypertension
2,2,10000826,28289260,1,5723,9,5723,Portal hypertension
3,3,10005866,26158160,4,5723,9,5723,Portal hypertension
4,4,10008924,23676183,7,5723,9,5723,Portal hypertension
...,...,...,...,...,...,...,...,...
4756321,4756321,19990427,29695607,24,T24011A,10,T24011A,"Burn of unspecified degree of right thigh, ini..."
4756322,4756322,19996016,28015466,4,O30093,10,O30093,"Twin pregnancy, unable to determine number of ..."
4756323,4756323,19996783,25894657,20,H353131,10,H353131,"Nonexudative age-related macular degeneration,..."
4756324,4756324,19997062,20096107,1,K8036,10,K8036,Calculus of bile duct with acute and chronic c...


In [13]:
diag_d = (diag
 .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version']))

In [37]:
d_diag

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [32]:
mappingicd9_icd10 = pd.read_csv("ICD9_to_ICD10_mapping.txt", delimiter='\t')

In [35]:
mappingicd9_icd10.columns
mappingicd9_icd10.loc[mappingicd9_icd10['icd9cm']== '5723']

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
5589,ICD9,572.3,PORTAL HYPERTENSION,5723,K766,0


In [ ]:
(diag_d
 .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version']))